# Gonzalo Carretero Ursúa

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

from pyspark.sql import SparkSession

# Inicio contesto Spark.SQL 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql import Row

In [2]:
from pyspark import SparkConf
conf = SparkConf()
conf.set("spark.driver.memory", "5g")

spark = SparkSession.builder \
            .config(conf=conf) \
            .master("local[*]") \
            .getOrCreate()

In [3]:
new = "BDpracticafinalCSV.csv" 
new = sc.textFile(new)
new.take(2)

['Order;MS Zoning;Lot Frontage;Lot Area;Land Contour;Land Slope;Overall Qual;Year Built;Year Remod/Add;Mas Vnr Type;Exter Qual;Bsmt Cond;Total Bsmt SF;Heating QC;Central Air;Full Bath;Half Bath;Bedroom AbvGr;Kitchen AbvGr;Kitchen Qual;TotRms AbvGrd;Garage Cars;Garage Area;Garage Cond;Pool Area;Pool QC;Mo Sold;Yr Sold;SalePrice',
 '1;RL;141;31770;Lvl;Gtl;6;1960;1960;Stone;TA;Gd;1080;Fa;Y;1;0;3;1;TA;7;2;528;TA;0;Sp;5;2010;215000']

In [4]:
header = new.take(1)[0]
raw_desarrollo = new \
    .filter(lambda line: line!=header)

In [5]:
new = raw_desarrollo.map(lambda x: x.split(";"))

In [6]:
def cambio(x):  # Funcion para cambiar los valores rL a RL y Rl a RL
    if x == 'rL':
        return 'RL'
    elif x == 'Rl':
        return 'RL'
    elif x == 'RL':
        return 'RL'
    elif x == 'RH':
        return 'RH'
    elif x == 'FV':
        return 'FV'
    elif x == 'C':
        return 'C'
    elif x == 'RM':
        return 'RM'
    elif x == 'I':
        return 'I'
    elif x == 'A':
        return 'A'

In [7]:
ruebanew = new.map(lambda x: cambio(x[1]))
ruebanew.take(5)

['RL', 'RH', 'RL', 'RL', 'RL']

Los cambios de int y float se iran realizando por apartado, para las variables que sean necesarias en la práctica.

In [8]:
new = new.map(lambda x: (x[0], cambio(x[1]), x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10],
                        x[11], x[12], x[13], x[14], x[15], x[16], x[17], x[18], x[19], x[20], x[21],
                         x[22], x[23], x[24], x[25], x[26], x[27], x[28]))

In [9]:
new = new.filter(lambda x: not 'A'  in x[1])\
         .filter(lambda x: not 'I'  in x[1])\
         .filter(lambda x: not 'C'  in x[1])\
         .filter(lambda x: not 'FV'  in x[1])

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Comprobar si existen viviendas duplicadas en la lista. Para ellos aplicamos un disctinct que nos eliminará las filas repetidas en función de la primera variable (Order: Variable de identificación) y después un count(). En el caso de que haya diferencia significará que existen viviendas repetidas.

In [257]:
new.map(lambda x: x[0]).distinct().count()

2762

In [258]:
new.map(lambda x: x[0]).count()

2768

Se observa que existe una diferencia de 6 viviendas aplicando el distinct y el count, esto significa que existen 6 viviendas repetidas en nuestro dataset.

Al ser una variable identificadora para cada vivienda no tiene sentido que se repitan, ya que un número identificaría 2 viviendas, lo cuál nos daría problema para acceder a los datos.

En primer lugar, se identifican que viviendas están repetidas.

In [259]:
aa = new.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
aa.filter(lambda x: x[1] != 1).collect()

[('2898', 2), ('2899', 2), ('2900', 2), ('2901', 2), ('2930', 2), ('2929', 2)]

Comprobamos que realmente son viviendas repetidas en el dataset y no un problema de identificación.

In [264]:
new.filter(lambda x: '2898' in x[0]).collect() # Comprobarlo para los 6 casos.

[('2898',
  'RL',
  '',
  '11170',
  'Lvl',
  'Gtl',
  '7',
  '1990',
  '1991',
  'None',
  'TA',
  'TA',
  '1216',
  'Ex',
  'Y',
  '2',
  '1',
  '4',
  '1',
  'TA',
  '8',
  '2',
  '693',
  'TA',
  '0',
  'Sp',
  '4',
  '2006',
  '250000'),
 ('2898',
  'RL',
  '',
  '11170',
  'Lvl',
  'Gtl',
  '7',
  '1990',
  '1991',
  'None',
  'TA',
  'TA',
  '1216',
  'Ex',
  'Y',
  '2',
  '1',
  '4',
  '1',
  'TA',
  '8',
  '2',
  '693',
  'TA',
  '0',
  'Sp',
  '4',
  '2006',
  '250000')]

Confirmamos que es un error de duplicación, por tanto, se eliminan los valores duplicados.

In [10]:
def get_key(x): # Esta función ha sido encontrada en internet, no es propia.
    return "{0}".format(x[0])

In [11]:
m = new.map(lambda x: (get_key(x),x))
m = m.reduceByKey(lambda x,y: (x))

In [12]:
m = m.map(lambda x: x[1])

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Calcular el número de viviendas que hay por cada una de las zonas (MS Zoning) y obtener el precio medio de esos inmuebles agrupados por zona.

A continuación se muestra el número de viviendas por cada una de las zonas.

In [16]:
m.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a + b).collect()

[('RL', 2273), ('RH', 27), ('RM', 462)]

In [13]:
preg2 = m.map(lambda x: (x[1], x[28]))
preg2.take(1)

[('RL', '215000')]

Media del precio de venta por cada una de las zonas.

In [304]:
yy = preg2.map(lambda x: (x[0], float(x[1])))
yy2 = yy.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yy6 = yy.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yy2.join(yy6).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.collect()

[('RL', 191283.25164980203),
 ('RH', 136419.77777777778),
 ('RM', 126781.39393939394)]

## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Year Built: 7

Total Bsmt SF: 12

Heating QC: 13

Se pide calcular la media de Total Bsmt SF (Superficie del sótano en pies cuadrados) por cada década y investigar en que decada de construcción se construyeron las viviendas más aptas ara el frio.

In [627]:
preg3 = m.map(lambda x: (x[7], (int(x[12]))))

In [628]:
preg3.map(lambda x: x[0]).min()

'1872'

In [629]:
preg3.map(lambda x: x[0]).max()

'2010'

In [630]:
preg3.take(5)

[('1960', 1080), ('1968', 2110), ('1992', 1280), ('1995', 1595), ('1999', 994)]

In [631]:
def get_decade(x):
    return x[0:3] + "0s"

In [632]:
preg3 = preg3.map(lambda x: (get_decade(x[0]), x[1]))

In [633]:
preg3.take(5)

[('1960s', 1080),
 ('1960s', 2110),
 ('1990s', 1280),
 ('1990s', 1595),
 ('1990s', 994)]

A continuación se muestra la media de pies de sótano construídos. Las décadas se ordenan de más pies construídos a menos.

In [637]:
yy3 = preg3.map(lambda x: (x[0], float(x[1])))
yyaa = yy3.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb = yy3.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa.join(yybb).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.sortBy(lambda x: x[0], ascending = False).collect()

[('2010s', 1539.0),
 ('2000s', 1348.469079939668),
 ('1990s', 1138.8083067092653),
 ('1980s', 1086.0),
 ('1970s', 952.7225274725274),
 ('1960s', 1082.672268907563),
 ('1950s', 968.2955223880597),
 ('1940s', 706.9261744966443),
 ('1930s', 769.9158878504672),
 ('1920s', 833.9368421052632),
 ('1910s', 788.3009708737864),
 ('1900s', 720.1388888888889),
 ('1890s', 885.5833333333334),
 ('1880s', 836.0),
 ('1870s', 761.0)]

In [661]:
preg3_2 = m.map(lambda x: (get_decade(x[7]), x[13]))

In [662]:
preg3_2.take(5)

[('1960s', 'Fa'),
 ('1960s', 'Ex'),
 ('1990s', 'Ex'),
 ('1990s', 'Ex'),
 ('1990s', 'Gd')]

In [685]:
pru = preg3_2.groupBy(lambda x: (x[0],x[1])).mapValues(list)

In [686]:
pru.collect()

[(('1960s', 'Fa'),
  [('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa'),
   ('1960s', 'Fa')]),
 (('1990s', 'Ex'),
  [('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),
   ('1990s', 'Ex'),


In [742]:
preg3_2.filter(lambda x: x[1] == 'Ex')\
.map(lambda x: (x[0],1))\
.reduceByKey(lambda a,b: a + b).collect()

[('1960s', 88),
 ('1950s', 98),
 ('1940s', 56),
 ('1900s', 12),
 ('1880s', 2),
 ('1990s', 233),
 ('2000s', 638),
 ('1970s', 68),
 ('1910s', 38),
 ('1980s', 28),
 ('1930s', 40),
 ('1920s', 51),
 ('1890s', 4),
 ('2010s', 2)]

La década con más viviendas en categoría "Excelent" corresponde a la década de 2000 a 2010.

In [743]:
preg3_2.filter(lambda x: x[1] == 'Gd')\
.map(lambda x: (x[0],1))\
.reduceByKey(lambda a,b: a + b).collect()

[('1960s', 79),
 ('1950s', 75),
 ('1880s', 3),
 ('1900s', 11),
 ('1870s', 1),
 ('1940s', 26),
 ('1990s', 72),
 ('1980s', 34),
 ('1970s', 41),
 ('1920s', 43),
 ('1910s', 28),
 ('1930s', 22),
 ('2000s', 25),
 ('1890s', 4)]

In [744]:
preg3_2.filter(lambda x: x[1] == 'TA')\
.map(lambda x: (x[0],1))\
.reduceByKey(lambda a,b: a + b).collect()

[('1960s', 180),
 ('1950s', 143),
 ('1940s', 54),
 ('1900s', 12),
 ('1880s', 3),
 ('1870s', 1),
 ('1970s', 243),
 ('1980s', 58),
 ('1920s', 75),
 ('1910s', 28),
 ('1930s', 41),
 ('1890s', 4),
 ('1990s', 8)]

In [745]:
preg3_2.filter(lambda x: x[1] == 'Fa')\
.map(lambda x: (x[0],1))\
.reduceByKey(lambda a,b: a + b).collect()

[('1960s', 10),
 ('1950s', 19),
 ('1940s', 12),
 ('1900s', 1),
 ('1870s', 1),
 ('1930s', 4),
 ('1920s', 20),
 ('1970s', 12),
 ('1910s', 9)]

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Obtener las 10 viviendas con mayor precio por metro cuadrado vendidas en 2009.

In [732]:
preg4 = m.map(lambda x: (x[0], (int(x[3])*0.093), x[27],float(x[28])))

In [733]:
preg4 = preg4.filter(lambda x: x[2] == '2009')

In [734]:
preg4.take(5)

[('342', 1209.744, '2009', 124500.0),
 ('344', 1288.98, '2009', 345000.0),
 ('345', 946.647, '2009', 189500.0),
 ('346', 1096.656, '2009', 270000.0),
 ('347', 781.2, '2009', 189000.0)]

In [735]:
preg4.map(lambda x: (x[0], (x[3]/x[1])))\
.sortBy(lambda x: x[1], ascending = False)\
     .takeOrdered(10, lambda x: len(x))

[('936', 1025.6410256410256),
 ('934', 989.9988437969708),
 ('464', 820.2006409891715),
 ('935', 807.1186531124097),
 ('408', 764.8489503328212),
 ('407', 755.2483358934971),
 ('405', 716.8458781362007),
 ('933', 713.536722322531),
 ('403', 710.4454685099846),
 ('411', 700.339558573854)]

## Media anual por zonas del precio de venta y metros cuadrados.

Se pide calcular el precio medio de venta por zona y los metros cuadrados medios por zona. En ambos casos agrupados por año.

MS Zoning: 2
sales price: 28
metros cuadrados: 3

1 pie = 0,093 metros

In [461]:
preg5 = m.map(lambda x: (x[1], x[7], float(x[3]), float(x[28])))
preg5 = preg5.map(lambda x: (x[0], x[1], (x[2]*0.093), x[3]))

In [462]:
prueba_preg5 = preg5.groupBy(lambda x: (x[0], x[1])).mapValues(list)
prueba_preg5.take(1)

[(('RL', '1995'),
  [('RL', '1995', 501.177, 236500.0),
   ('RL', '1995', 1368.96, 410000.0),
   ('RL', '1995', 745.209, 239500.0),
   ('RL', '1995', 1407.834, 403000.0),
   ('RL', '1995', 844.905, 260000.0),
   ('RL', '1995', 914.004, 239500.0),
   ('RL', '1995', 756.927, 165000.0),
   ('RL', '1995', 970.641, 350000.0),
   ('RL', '1995', 786.129, 182000.0),
   ('RL', '1995', 1028.952, 217500.0),
   ('RL', '1995', 908.703, 260000.0),
   ('RL', '1995', 1065.222, 225000.0),
   ('RL', '1995', 885.825, 144000.0),
   ('RL', '1995', 963.852, 168000.0),
   ('RL', '1995', 925.35, 290000.0),
   ('RL', '1995', 1139.901, 290000.0),
   ('RL', '1995', 1211.883, 187500.0),
   ('RL', '1995', 3325.68, 625000.0),
   ('RL', '1995', 888.057, 270000.0),
   ('RL', '1995', 1046.25, 237000.0),
   ('RL', '1995', 793.29, 168500.0),
   ('RL', '1995', 744.0, 188000.0),
   ('RL', '1995', 1109.397, 290000.0),
   ('RL', '1995', 732.375, 176000.0),
   ('RL', '1995', 1046.157, 177500.0),
   ('RL', '1995', 930.0, 2400

ZONA RL residencia

In [452]:
RL_PREG5 = preg5.filter(lambda x: x[0] == 'RL')

In [457]:
RL_PREG5_M2 = RL_PREG5.map(lambda x: (x[1], x[2]))
yyaa_RL = RL_PREG5_M2.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RL = RL_PREG5_M2.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RL.join(yybb_RL).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2)          

[('1995', 1073.0910000000001), ('1990', 1018.4332105263157)]

In [459]:
RL_PREG5_cost = RL_PREG5.map(lambda x: (x[1], x[3]))
yyaa_RL = RL_PREG5_cost.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RL = RL_PREG5_cost.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RL.join(yybb_RL).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2) 

[('1995', 247500.0), ('1990', 215951.57894736843)]

RM

In [465]:
RM_PREG5 = preg5.filter(lambda x: x[0] == 'RM')

In [466]:
RM_PREG5_M2 = RM_PREG5.map(lambda x: (x[1], x[2]))
yyaa_RM = RM_PREG5_M2.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RM = RM_PREG5_M2.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RM.join(yybb_RM).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2) 

[('1978', 830.49), ('1927', 669.3365)]

In [468]:
RM_PREG5_cost = RM_PREG5.map(lambda x: (x[1], x[3]))
yyaa_RM  = RM_PREG5_cost.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RM = RM_PREG5_cost.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RM.join(yybb_RM).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2) 

[('1978', 112000.0), ('1927', 123333.33333333333)]

RH

In [469]:
RH_PREG5 = preg5.filter(lambda x: x[0] == 'RH')

In [470]:
RH_PREG5_M2 = RH_PREG5.map(lambda x: (x[1], x[2]))
yyaa_RH = RH_PREG5_M2.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RH = RH_PREG5_M2.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RH.join(yybb_RH).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2) 

[('1920', 911.4), ('1925', 873.4559999999999)]

In [471]:
RH_PREG5_cost = RH_PREG5.map(lambda x: (x[1], x[3]))
yyaa_RH  = RH_PREG5_cost.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
yybb_RH = RH_PREG5_cost.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)
media2 = yyaa_RH.join(yybb_RH).map(lambda x:(x[0],(x[1][0]/x[1][1])))
media2.take(2) 

[('1920', 67500.0), ('1925', 121800.0)]

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Se pide calcular el sumatorio de los precios de venta de las viviendas agrupadas respecto el tipo de revestimiento.

(Mas Vnr Type): 9

" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "

In [17]:
PREG6 = m.map(lambda x: (x[9], float(x[28])))
PREG6.take(5)

[('Stone', 215000.0),
 ('None', 244000.0),
 ('None', 191500.0),
 ('None', 236500.0),
 ('None', 189000.0)]

In [484]:
PREG6_STONE = PREG6.filter(lambda x: x[0] == 'Stone')\
.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
PREG6_STONE.collect()

[('Stone', 60175477.0)]

In [18]:
PREG6_LADRILLO = PREG6.filter(lambda x: not 'CBlock' in x[0])\
                   .filter(lambda x: not 'None' in x[0])\
                   .filter(lambda x: not 'Stone' in x[0])\
                   .map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b: a + b)
PREG6_LADRILLO.collect()

[('', 3831658.0), ('BrkFace', 178912561.0), ('BrkCmn', 3504975.0)]

In [19]:
ladrillo = 178912561.0 + 3504975.0

print("El sumatorio total de la recaudación con revestimiento de ladrillo es" ,format(ladrillo))

El sumatorio total de la recaudación con revestimiento de ladrillo es 182417536.0


In [20]:
diferencia = 60175477.0 - 182417536.0

print("La diferencia entre el total recaudado por ladrillo y piedra es" ,format(diferencia))

La diferencia entre el total recaudado por ladrillo y piedra es -122242059.0


Esto significa que se han constuido más viviendas con revestimiento de ladrillo.

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

Calcular el precio medio y los cuartiles para las viviendas con 1 cocina y 1 plaza de garaje, y para las viviendas con 2 cocinas y más de 1 plaza de garaje.

Kitchen AbvGr: 18
Garage Cars: 21

In [21]:
import statistics as stats 

In [22]:
last_q = m.map(lambda x: (x[18], x[21], float(x[28])))

In [24]:
cocina1 = last_q.filter(lambda x: x[0] == '1')\
                .filter(lambda x: x[1] == '1')

In [25]:
media1 = cocina1.map(lambda x: x[2]).mean()

print("La media de las viviendas con una cocina y un garaje" ,format(media1))

La media de las viviendas con una cocina y un garaje 128121.99337748342


In [570]:
uno_uno = cocina1.map(lambda x: x[2])

In [580]:
uno_uno1 = uno_uno.collect()
uno_uno1.sort()

In [582]:
uno_uno1[int(len(uno_uno1)*.25)]

111500.0

In [583]:
uno_uno1[int(len(uno_uno1)*.50)]

128000.0

In [584]:
uno_uno1[int(len(uno_uno1)*.75)]

142125.0

In [26]:
cocina2 = last_q.filter(lambda x: x[0] == '2')\
                .filter(lambda x: x[1] != '1')

In [27]:
media2 = cocina2.map(lambda x: x[2]).mean()

print("La media de las viviendas con dos cocina y más de una plaza de garaje" ,format(media2))

La media de las viviendas con dos cocina y más de una plaza de garaje 137004.8119658119


In [585]:
no_uno_uno = cocina2.map(lambda x: x[2])

In [586]:
no_uno_uno2 = no_uno_uno.collect()
no_uno_uno2.sort()

In [587]:
no_uno_uno2[int(len(no_uno_uno2)*.25)]

111500.0

In [588]:
no_uno_uno2[int(len(no_uno_uno2)*.50)]

136000.0

In [589]:
no_uno_uno2[int(len(no_uno_uno2)*.75)]

153337.0

In [32]:
print("La diferencia de medias es", format(media1-media2))

La diferencia de medias es -8882.81858832849


Esto significa que en media son más caras las viviendas con 2 cocinas y más de una plaza de garaje. En los cuartiles tambien se observa diferencia. Las viviendas con 1 cocina y 1 plaza de garaje tienen unos cuartiles más reducidos.

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

In [33]:
compl1 = m.map(lambda x: (x[0], x[21], int(x[28])))

In [50]:
compl1_pr = m.map(lambda x: (x[0], int(x[21], x[28])))

In [54]:
compl1.collect() # Tengo problemas en la variable 21 con NAs o algún formato. No consigo
# encontrar cuales son los registros con problemas para tratarlos.

[('1', '2', 215000),
 ('4', '2', 244000),
 ('8', '2', 191500),
 ('9', '2', 236500),
 ('10', '2', 189000),
 ('12', '2', 185000),
 ('14', '2', 171500),
 ('16', '3', 538000),
 ('17', '2', 164000),
 ('19', '2', 141000),
 ('20', '2', 210000),
 ('21', '2', 190000),
 ('22', '2', 170000),
 ('24', '2', 149000),
 ('26', '1', 142000),
 ('29', '2', 184000),
 ('33', '2', 127500),
 ('34', '2', 149900),
 ('40', '3', 290941),
 ('44', '2', 214000),
 ('45', '3', 611657),
 ('48', '3', 320000),
 ('50', '2', 205000),
 ('53', '2', 160000),
 ('54', '2', 192000),
 ('56', '2', 216500),
 ('57', '2', 185088),
 ('60', '3', 333168),
 ('63', '2', 325000),
 ('64', '3', 290000),
 ('66', '3', 410000),
 ('73', '2', 233000),
 ('74', '2', 181000),
 ('77', '2', 189000),
 ('82', '2', 171000),
 ('83', '1', 67500),
 ('84', '2', 112000),
 ('86', '1', 138500),
 ('88', '1', 133000),
 ('91', '2', 190000),
 ('93', '3', 285000),
 ('107', '2', 155000),
 ('108', '2', 206000),
 ('111', '2', 218500),
 ('112', '2', 243500),
 ('113', '2

In [15]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import Row

In [18]:
De = compl1.map(lambda x: Row(
Vivienda= str(x[0]),
Garajes= x[1],
Precio= int(x[2])))
debtsql = sqlContext.createDataFrame(De)

In [28]:
bignine = debtsql.sort('Garajes').dropna()

In [29]:
df2 = bignine.selectExpr("cast(Garajes as int) Garajes","Precio")

In [31]:
df2.corr('Garajes', 'Precio')

0.6452399088308861

## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta

In [ ]:
sc.stop()